In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import pandas
import sqlite3
import seaborn as sns
from sklearn import *
from tqdm import tqdm
import joblib
import ml_metrics

/Users/riri/.virtualenvs/analysis3/lib/python3.4/site-packages/sklearn/lda.py:4: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 and will be removed in 0.19", DeprecationWarning)
/Users/riri/.virtualenvs/analysis3/lib/python3.4/site-packages/sklearn/qda.py:4: DeprecationWarning: qda.QDA has been moved to discriminant_analysis.QuadraticDiscriminantAnalysis in 0.17 and will be removed in 0.19.
  "in 0.17 and will be removed in 0.19.", DeprecationWarning)


In [3]:
con = sqlite3.connect('/tmp/data.sqlite3')

In [4]:
samples = pandas.read_sql('SELECT product_id, count(*) from data group by 1 having count(*) > 100', con=con)

In [27]:
class SelectiveClassifier:
    def __init__(self, field, value):
        self.field = field
        self.value = value
    
    def fit(self):
        con = sqlite3.connect('/tmp/data.sqlite3')
        try:
            query = '''
                SELECT week_num, 
                       sales_channel,
                       route_id,
                       client_id,
                       sales_depo,
                       product_id,
                       adjusted_demand
                  FROM data
                 WHERE adjusted_demand is not null
                       AND week_num < 8
                       AND %s = %d
            ''' % (self.field, self.value)
            data = pandas.read_sql(query, con=con).drop(self.field, axis=1)
            X = data.drop('adjusted_demand', axis=1)
            y = data.adjusted_demand
            self.cls = ensemble.ExtraTreesRegressor(n_estimators=100, max_leaf_nodes=100)
            self.cls.fit(X, y)
        finally:
            con.close()
        
    def score(self):
        y, pred_y = self.predict_test()
        return ml_metrics.rmsle(y, pred_y)

    def predict_test(self):
        con = sqlite3.connect('/tmp/data.sqlite3')
        try:
            query = '''
                SELECT week_num, 
                       sales_channel,
                       route_id,
                       client_id,
                       sales_depo,
                       product_id,
                       adjusted_demand
                  FROM data
                 WHERE adjusted_demand is not null
                       AND week_num >= 8
                       AND %s = %d
            ''' % (self.field, self.value)
            data = pandas.read_sql(query, con=con).drop(self.field, axis=1)
            X = data.drop('adjusted_demand', axis=1)
            y = data.adjusted_demand
            pred_y = self.cls.predict(X)
            return y, pred_y
        finally:
            con.close()
            
def fitscore(prod):
    d = SelectiveClassifier('product_id', prod)
    d.fit()
    return d.score()

In [30]:
scores = joblib.Parallel(n_jobs=-1, verbose=10)([joblib.delayed(fitscore)(prod) 
                                                 for prod in samples.product_id])

[Parallel(n_jobs=-1)]: Batch computation too fast (0.1980s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Batch computation too slow (2.10s.) Setting batch_size=1.
[Parallel(n_jobs=-1)]: Done  35 tasks      | elapsed:   12.2s
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   26.2s
[Parallel(n_jobs=-1)]: Done  61 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done  73 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done  86 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done  99 tasks      | elapsed:  4.1min


JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/usr/local/Cellar/python3/3.4.3_2/Frameworks/Python.framework/Versions/3.4/lib/python3.4/runpy.py in _run_module_as_main(mod_name='ipykernel.__main__', alter_argv=1)
    165         sys.exit(msg)
    166     main_globals = sys.modules["__main__"].__dict__
    167     if alter_argv:
    168         sys.argv[0] = mod_spec.origin
    169     return _run_code(code, main_globals, None,
--> 170                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel.__main__', loader=<_f...b/python3.4/site-packages/ipykernel/__main__.py')
    171 
    172 def run_module(mod_name, init_globals=None,
    173                run_name=None, alter_sys=False):
    174     """Execute a module's code without importing it

.....................................................

In [ ]:
np.mean(scores)